In [48]:
import os
import pandas as pd
import openpyxl
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Set directory path to current directory
dir_path = os.path.abspath('')
elec_path = os.path.abspath('../control-variables')

# dir_path = os.path.abspath(os.path.join(os.path.dirname(__file__)))
                                        # , ".."))                                    
aid = os.path.join(dir_path, 'aid_oecd.xlsx')
elec = os.path.join(elec_path, 'inde-variable-data.xlsx')

In [49]:
# Import the OECD foreign aid data into dataframe
aid_df = pd.read_excel(aid, index_col=0, sheet_name="transactions")
aid_df = aid_df.sort_values("recipient_name", ascending=True)

# Import the electrification rate data into dataframe
elec_df = pd.read_excel(elec, index_col=0, sheet_name="elec_rate")
elec_df = elec_df.sort_values("Country Name", ascending=True)

In [48]:
# print(aid_df.info())

In [50]:
# Fill in NA values with 0s.
_ = elec_df.fillna(0, inplace=True)

# Stack electrification rate by country and year
stacked_elec_df = elec_df.stack()

In [31]:
# elec_df.transpose().plot(subplots=True, grid=True, style='b', figsize=(60, 60))

In [53]:
# Calculate sector aid by country per year
sector_aid = aid_df.groupby(['recipient_name', 'year_name', 'lvl_1_sector_name'])[['usd_disbursement']].sum()

In [54]:
# Organize sub-sector into columns
sub_sec_sum = sector_aid.reset_index().pivot_table(
                   index=['recipient_name', 'year_name'],
                   columns='lvl_1_sector_name',
                   values='usd_disbursement')

# Fill in NA values with 0s.
_ = sub_sec_sum.fillna(0, inplace=True)

In [55]:
#### Don't run me!!!!!!!!!!!!!!!

output = os.path.join(dir_path, 'results_v1.xlsx')

sub_sec_sum.to_excel(output, sheet_name='aid')

In [56]:
# Append the transpised electricity data to new excel sheet
# if_sheet_exists para needs to be set to overlay to be able to write multiple df into one single sheet
with pd.ExcelWriter(output,
                    engine='openpyxl',
                    mode='a',
                    if_sheeet_exists="overlay",
) as writer:
    writer.book = openpyxl.load_workbook(output)
    stacked_elec_df.to_excel(writer, sheet_name='elec_rate')

In [112]:
# for name, group in data_df.groupby('recipient_name'):

#     print(name)
y1=angola_df['elec_rate']
X1=angola_df[['Energy distribution',
             'Energy generation, non-renewable sources',
             'Energy Policy',
             'Energy generation, renewable sources'                  
            ]]

#     pooled_X = sm.add_constant(pooled_X)
    
res = sm.OLS(y1, X1).fit()

print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:              elec_rate   R-squared (uncentered):                   0.569
Model:                            OLS   Adj. R-squared (uncentered):              0.446
Method:                 Least Squares   F-statistic:                              4.620
Date:                Tue, 19 Apr 2022   Prob (F-statistic):                      0.0138
Time:                        12:58:32   Log-Likelihood:                         -82.682
No. Observations:                  18   AIC:                                      173.4
Df Residuals:                      14   BIC:                                      176.9
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------

/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [58]:
# Getting to the path for the 48 counries that received aid
country_aid = os.path.join(dir_path, 'results_v1.xlsx')

In [69]:
# Import the 48 aid countries' data into dataframe
country_aid_df = pd.read_excel(country_aid, index_col=0, sheet_name="aid_data")

# Fill the empty country names up
country_aid_df.index = pd.Series(country_aid_df.index).fillna(method='ffill')

# Sort country names so they match the elec data
country_aid_df = country_aid_df.sort_values("recipient_name", ascending=True)

country_aid_df.tail()

,year_name,Energy (unspecified),Energy Policy,Energy distribution,"Energy generation, non-renewable sources","Energy generation, renewable sources",Hybrid energy plants,Nuclear energy plants
recipient_name,,,,,,,,
Zimbabwe,2003,0,2351,0,0,100662,0,0
Zimbabwe,2002,0,87075,862330,40365,112446,0,0
Zimbabwe,2018,0,0,0,0,757957,0,0
Zimbabwe,2009,0,522306,0,0,63230,0,0
Zimbabwe,2019,0,346881,77434,0,853224,0,0


In [70]:
country_aid_df.reset_index()

,recipient_name,year_name,Energy (unspecified),Energy Policy,Energy distribution,"Energy generation, non-renewable sources","Energy generation, renewable sources",Hybrid energy plants,Nuclear energy plants
0,Angola,2002,0,4504687,-23027,-23027,0,0,0
1,Angola,2019,0,0,1482982,0,17188564,0,0
2,Angola,2018,0,188267,0,0,3862176,0,0
3,Angola,2017,0,489240,0,0,5520532,0,0
4,Angola,2016,0,488970,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
816,Zimbabwe,2003,0,2351,0,0,100662,0,0
817,Zimbabwe,2002,0,87075,862330,40365,112446,0,0
818,Zimbabwe,2018,0,0,0,0,757957,0,0
819,Zimbabwe,2009,0,522306,0,0,63230,0,0


In [43]:
# Write the organized data

with pd.ExcelWriter(output,
                    engine='openpyxl',
                    mode='a',
                    if_sheeet_exists="overlay",
) as writer:
    writer.book = openpyxl.load_workbook(output)
    aid48_df.to_excel(writer, sheet_name='please_fucking_work')
    
please_work = pd.read_excel(country48, sheet_name="please_fucking_work")

In [81]:
# Group the aid dataframe by year
country_aid_df['year_name'] = pd.Categorical(country_aid_df['year_name'], categories=country_aid_df['year_name'].unique())

complete_data = country_aid_df.groupby(['recipient_name', 'year_name'], as_index=False).first()

In [90]:
# Fill the empty cell with 0s
# complete_data = complete_data.fillna(0, limit=4, inplace=True)

complete_data['Energy distribution']=complete_data['Energy distribution'].fillna(0)
complete_data['Energy Policy']=complete_data['Energy Policy'].fillna(0)
complete_data['Energy generation, non-renewable sources']=complete_data['Energy generation, non-renewable sources'].fillna(0)
complete_data['Energy generation, renewable sources']=complete_data['Energy generation, renewable sources'].fillna(0)

In [91]:
output = os.path.join(dir_path, 'results_v1.xlsx')

with pd.ExcelWriter(output,
                    engine='openpyxl',
                    mode='a',
                    if_sheeet_exists="overlay",
) as writer:
    writer.book = openpyxl.load_workbook(output)
    complete_data.to_excel(writer, sheet_name='country_aid')